In [206]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import os
import torch # type: ignore
import torch.nn as nn # type: ignore
from torch.utils.data import Dataset, DataLoader # type: ignore
import torch.optim as optim # type: ignore
from os.path import join
import datetime
import json

%matplotlib inline

PATH_TO_DATA = os.path.join("../data", "Task3")

In [207]:
full_df = pd.read_csv(os.path.join(PATH_TO_DATA, "housing.csv"))


In [235]:
full_df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [208]:
train_df, test_df = train_test_split(full_df,shuffle = True, test_size = 0.25, random_state=17)
train_df=train_df.copy()
test_df=test_df.copy()
print(train_df.shape)
print(test_df.shape)

(15480, 10)
(5160, 10)


In [209]:
numerical_features = list(train_df.columns)
numerical_features.remove("ocean_proximity")
numerical_features.remove("median_house_value")

In [210]:
max_house_age = train_df["housing_median_age"].max()
train_df["age_clipped"] = train_df["housing_median_age"] == max_house_age
test_df["age_clipped"] = test_df["housing_median_age"] == max_house_age

In [211]:
train_df["median_house_value_log"] = np.log1p(train_df["median_house_value"])
test_df["median_house_value_log"] = np.log1p(test_df["median_house_value"])

skewed_features = [
    "households",
    "median_income",
    "population",
    "total_bedrooms",
    "total_rooms",
]
log_numerical_features = []
for f in skewed_features:
    train_df[f + "_log"] = np.log1p(train_df[f])
    test_df[f + "_log"] = np.log1p(test_df[f])
    log_numerical_features.append(f + "_log")

In [212]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin = LinearRegression()

# we will train our model based on all numerical non-target features with not NaN total_bedrooms
appropriate_columns = train_df.drop(
    [
        "median_house_value",
        "median_house_value_log",
        "ocean_proximity",
        "total_bedrooms_log",
    ],
    axis=1,
)
train_data = appropriate_columns[~pd.isnull(train_df).any(axis=1)]
temp_train, temp_valid = train_test_split(
    train_data, shuffle=True, test_size=0.25, random_state=17
)

lin.fit(temp_train.drop(["total_bedrooms"], axis=1), temp_train["total_bedrooms"])
np.sqrt(
    mean_squared_error(
        lin.predict(temp_valid.drop(["total_bedrooms"], axis=1)),
        temp_valid["total_bedrooms"],
    )
)

64.53336471582229

In [213]:
lin.fit(train_data.drop(["total_bedrooms"], axis=1), train_data["total_bedrooms"])
train_df["total_bedrooms_is_nan"] = pd.isnull(train_df).any(axis=1).astype(int)
test_df["total_bedrooms_is_nan"] = pd.isnull(test_df).any(axis=1).astype(int)

train_df["total_bedrooms"].loc[pd.isnull(train_df).any(axis=1)] = lin.predict(
    train_df.drop(
        [
            "median_house_value",
            "median_house_value_log",
            "total_bedrooms",
            "total_bedrooms_log",
            "ocean_proximity",
            "total_bedrooms_is_nan",
        ],
        axis=1,
    )[pd.isnull(train_df).any(axis=1)]
)

test_df["total_bedrooms"].loc[pd.isnull(test_df).any(axis=1)] = lin.predict(
    test_df.drop(
        [
            "median_house_value",
            "median_house_value_log",
            "total_bedrooms",
            "total_bedrooms_log",
            "ocean_proximity",
            "total_bedrooms_is_nan",
        ],
        axis=1,
    )[pd.isnull(test_df).any(axis=1)]
)

# linear regression can lead to negative predictions, let's change it
test_df["total_bedrooms"] = test_df["total_bedrooms"].apply(lambda x: max(x, 0))
train_df["total_bedrooms"] = train_df["total_bedrooms"].apply(lambda x: max(x, 0))

C:\Users\patri\AppData\Local\Temp\ipykernel_19580\945881367.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_df["total_bedrooms"].loc[pd.isnull(train_df).any(axis=1)] = lin.predict(
C:\Users\patri\AppData\Local\Temp\ipykernel_19580\

In [214]:
train_df["total_bedrooms_log"] = np.log1p(train_df["total_bedrooms"])
test_df["total_bedrooms_log"] = np.log1p(test_df["total_bedrooms"])

In [215]:
ocean_proximity_dummies = pd.get_dummies(
    pd.concat([train_df["ocean_proximity"], test_df["ocean_proximity"]]),
    drop_first=True,
)
dummies_names = list(ocean_proximity_dummies.columns)
train_df = pd.concat([train_df, ocean_proximity_dummies[: train_df.shape[0]]], axis=1)
test_df = pd.concat([test_df, ocean_proximity_dummies[train_df.shape[0] :]], axis=1)

train_df = train_df.drop(["ocean_proximity"], axis=1)
test_df = test_df.drop(["ocean_proximity"], axis=1)


In [216]:
sf_coord = [-122.4194, 37.7749]
la_coord = [-118.2437, 34.0522]

train_df["distance_to_SF"] = np.sqrt(
    (train_df["longitude"] - sf_coord[0]) ** 2
    + (train_df["latitude"] - sf_coord[1]) ** 2
)
test_df["distance_to_SF"] = np.sqrt(
    (test_df["longitude"] - sf_coord[0]) ** 2 + (test_df["latitude"] - sf_coord[1]) ** 2
)

train_df["distance_to_LA"] = np.sqrt(
    (train_df["longitude"] - la_coord[0]) ** 2
    + (train_df["latitude"] - la_coord[1]) ** 2
)
test_df["distance_to_LA"] = np.sqrt(
    (test_df["longitude"] - la_coord[0]) ** 2 + (test_df["latitude"] - la_coord[1]) ** 2
)

In [217]:
columns_for_ml = [
    "longitude",
    "latitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income",
    "median_house_value",
    "age_clipped",
    "total_bedrooms_is_nan",
    "INLAND",
    "ISLAND",
    "NEAR BAY",
    "NEAR OCEAN",
    "distance_to_SF",
    "distance_to_LA",
]
train_df=train_df[columns_for_ml]
test_df=test_df[columns_for_ml]

In [218]:
train_df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'age_clipped', 'total_bedrooms_is_nan', 'INLAND',
       'ISLAND', 'NEAR BAY', 'NEAR OCEAN', 'distance_to_SF', 'distance_to_LA'],
      dtype='object')

In [219]:
dummies_names

['INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']

In [220]:
numerical_features

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [221]:
from sklearn.preprocessing import StandardScaler

features_to_scale = (
    numerical_features + ["distance_to_SF", "distance_to_LA"]
)

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(
    scaler.fit_transform(train_df[features_to_scale]),
    columns=features_to_scale,
    index=train_df.index,
)
X_test_scaled = pd.DataFrame(
    scaler.transform(test_df[features_to_scale]),
    columns=features_to_scale,
    index=test_df.index,
)

In [222]:
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=10, random_state=17, shuffle=True)
X = pd.concat(
    [train_df[dummies_names+['age_clipped']], X_train_scaled],
    axis=1,
    ignore_index=True,
)
y = train_df["median_house_value"].reset_index(drop=True)

In [223]:
X.describe()

,5,6,7,8,9,10,11,12,13,14
count,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04,1.548000e+04
mean,-4.980225e-16,-2.698043e-15,-3.304850e-17,1.101617e-17,1.858978e-17,4.131062e-18,1.262269e-18,8.904735e-17,1.836028e-17,-9.363741e-17
std,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00,1.000032e+00
min,-2.379534e+00,-1.441623e+00,-2.191505e+00,-1.221424e+00,-1.288736e+00,-1.267593e+00,-1.308833e+00,-1.769147e+00,-1.548059e+00,-1.095508e+00
25%,-1.114154e+00,-7.971526e-01,-8.452381e-01,-5.494133e-01,-5.774149e-01,-5.685615e-01,-5.779252e-01,-6.881189e-01,-1.067902e+00,-9.634331e-01
50%,5.298445e-01,-6.430401e-01,2.587558e-02,-2.333367e-01,-2.457856e-01,-2.310978e-01,-2.375387e-01,-1.778486e-01,5.525557e-01,-3.954593e-01
75%,7.801806e-01,9.728059e-01,6.594128e-01,2.371864e-01,2.660771e-01,2.697412e-01,2.849151e-01,4.618554e-01,7.875922e-01,1.056705e+00
max,2.542497e+00,2.948248e+00,1.847295e+00,1.363325e+01,1.173372e+01,3.058523e+01,1.201110e+01,5.813177e+00,2.172428e+00,2.981499e+00


In [224]:
X.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
3400,False,False,False,False,False,0.609553,-0.633700,0.105068,0.267195,-0.055940,0.247422,0.092293,0.424206,0.594737,-0.993357
9159,False,False,False,False,False,0.559735,-0.558979,-1.003622,4.990733,4.896872,5.005839,5.124210,0.419866,0.522846,-0.915681
10592,False,False,False,False,False,0.893517,-0.909234,-1.003622,0.353398,-0.084777,0.135827,-0.026446,1.157171,0.921515,-0.854024
4281,False,False,False,False,False,0.624499,-0.722431,-0.132509,-1.125025,-1.053231,-0.973875,-1.018581,-0.694198,0.653895,-1.062084
230,False,False,True,False,False,-1.308445,1.005496,0.896989,-0.731088,-0.673539,-0.674800,-0.694026,0.152030,-1.462103,1.152951
7132,False,False,False,False,False,0.724135,-0.750452,-0.686854,-0.019684,-0.260204,0.292953,-0.050194,0.903611,0.729682,-1.045329
10801,False,False,False,False,False,0.808826,-0.937255,0.025876,-0.529021,-0.611058,-0.856923,-0.746799,1.133901,0.887165,-0.884653
3022,True,False,False,False,False,0.061554,-0.222733,0.421836,0.370546,0.383829,0.321521,0.290192,-0.780164,0.031962,-0.420160
9099,True,False,False,False,False,0.823771,-0.470247,0.421836,-1.184811,-1.226255,-1.222955,-1.277169,-1.293716,0.644861,-0.823470
14109,False,False,False,True,False,1.227298,-1.348221,0.421836,-0.234727,0.138712,-0.044511,0.168814,-0.925638,1.372831,-0.383463


In [225]:
X.columns

RangeIndex(start=0, stop=15, step=1)

In [226]:
X.shape[1]

15

In [227]:
X_train_scaled.shape[1]

10

In [228]:
X.dtypes



0        bool
1        bool
2        bool
3        bool
4        bool
5     float64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
12    float64
13    float64
14    float64
dtype: object

In [229]:
torch.tensor(X.values.astype(np.float32))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.4242,  0.5947, -0.9934],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.4199,  0.5228, -0.9157],
        [ 0.0000,  0.0000,  0.0000,  ...,  1.1572,  0.9215, -0.8540],
        ...,
        [ 1.0000,  0.0000,  0.0000,  ..., -0.6775,  0.9901, -0.6775],
        [ 1.0000,  0.0000,  0.0000,  ..., -0.4163, -0.5236,  0.2462],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.1134,  0.8863, -0.8908]])

In [230]:
class NeuralNet(nn.Module):

    def __init__(
        self, input_dimension, output_dimension, n_hidden_layers, neurons, retrain_seed, output_activation=None
    ):
        super(NeuralNet, self).__init__()
        # Number of input dimensions n
        self.input_dimension = input_dimension
        # Number of output dimensions m
        self.output_dimension = output_dimension
        # Number of neurons per layer
        self.neurons = neurons
        # Number of hidden layers
        self.n_hidden_layers = n_hidden_layers
        # Activation function
        # self.activation = nn.Tanh()
        # self.activation = nn.LeakyReLU()
        self.activation=nn.ReLU()
        # self.output_activation=nn.ReLU()
        if output_activation is None:
            self.output_activation = nn.Identity()

        self.input_layer = nn.Linear(self.input_dimension, self.neurons)
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(self.neurons, self.neurons) for _ in range(n_hidden_layers - 1)]
        )
        self.output_layer = nn.Linear(self.neurons, self.output_dimension)
        self.retrain_seed = retrain_seed
        # Random Seed for weight initialization
        self.init_xavier()

    def forward(self, x):
        # The forward function performs the set of affine and non-linear transformations defining the network
        # (see equation above)
        x = self.activation(self.input_layer(x))
        for k, l in enumerate(self.hidden_layers):
            x = self.activation(l(x))
        return self.output_activation(self.output_layer(x))

    def init_xavier(self):
        torch.manual_seed(self.retrain_seed)

        def init_weights(m):
            if type(m) == nn.Linear and m.weight.requires_grad and m.bias.requires_grad:
                g = nn.init.calculate_gain("tanh")
                torch.nn.init.xavier_uniform_(m.weight, gain=g)
                # torch.nn.init.xavier_normal_(m.weight, gain=g)
                m.bias.data.fill_(0)

        self.apply(init_weights)

In [231]:
class CalHousing:
    def __init__(self, n_hidden_layers, n_neurons, train_df, target_df,X_valid,y_valid, seed):
        self.n_hidden_layers=n_hidden_layers
        self.n_neurons = n_neurons
        self.seed=seed
        self.device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        train_tensor = torch.tensor(train_df.values.astype(np.float32),dtype=torch.float32)
        target_tensor = torch.tensor(self.scale_targets(target_df), dtype=torch.float32)
        self.data = DataLoader(
            torch.utils.data.TensorDataset(train_tensor, target_tensor),
            batch_size=64,
            shuffle=False
        )
        self.model=NeuralNet(
            input_dimension=train_df.shape[1],
            output_dimension=1,
            n_hidden_layers=n_hidden_layers,
            neurons=n_neurons,
            retrain_seed=self.seed

        ).to(self.device)
        self.X=train_df
        self.y=target_df

        self.X_valid=X_valid
        self.y_valid=y_valid

        self.metadata_file=join("../logs","task_3" ,"metadata.json")

    def compute_loss(self, inputs, targets, verbose=True):
        preds=self.model(inputs)
        # targets=targets/10000
        res=targets-preds
        loss=torch.mean(res**2)
        # if verbose: print("Total loss: ", round(loss.item(), 4))
        return loss

    def save(self, loss_history):
        filename=join("..","models", "task_3", datetime.datetime.now().strftime("%m-%d %H:%M:%S")+".pt")
        salient_info={}
        salient_info["n_hidden_layers"]=self.n_hidden_layers
        salient_info["n_neurons"]=self.n_neurons
        salient_info["final_loss"]=loss_history[-1]
        salient_info["min_loss"]=min(loss_history)
        salient_info["model_path"]=filename
        salient_info["seed"]=self.seed

        torch.save(self.model.state_dict(),filename )
        with open(self.metadata_file, "a") as f:
            json.dump(salient_info, f)

    def scale_targets(self, target_df):
        y=target_df.values.astype(np.float32)
        self.scaler=StandardScaler()
        return(self.scaler.fit_transform(y.reshape(-1, 1)))
    
    # def validate(self, X,y):
    #     self.model.eval()
    #     with torch.no_grad():
    #         inputs=torch.tensor(X.values.astype(np.float32),dtype=torch.float32).to(self.device)
    #         targets=torch.tensor(self.scale_targets(y),dtype=torch.float32).to(self.device)
    #         loss=self.compute_loss(inputs, targets)
    #         return loss
    
    def RMSE(self, validation=True):
        if validation:
            X=self.X_valid
            y=self.y_valid
        else:
            X=self.X
            y=self.y
        self.model.eval()
        with torch.no_grad():
            inputs=torch.tensor(X.values.astype(np.float32),dtype=torch.float32).to(self.device)
            # targets=torch.tensor(self.scale_targets(y),dtype=torch.float32).to(self.device)
            preds=self.model(inputs)
            preds=self.scaler.inverse_transform(preds.cpu().numpy())
            loss=np.sqrt(np.mean((y.values-preds)**2))
            return loss



        

In [232]:
def fit(model: CalHousing, num_epochs, optimizer, verbose=True):
    history=list()
    device=model.device


    for epoch in range(num_epochs):
        if verbose: print("################################ ",epoch," ##################################")

        for inputs, targets in model.data:
            def closure():
                optimizer.zero_grad()
                loss=model.compute_loss(inputs.to(device), targets.to(device), verbose=True)
                loss.backward()
                history.append(loss.item())
                return loss
            optimizer.step(closure)
            



            # inputs = inputs.to(device)
            # targets = targets.to(device)
            # optimizer.zero_grad()
            # if epoch==378:
            #     pass
            # loss = model.compute_loss(inputs, targets, verbose=False)
            # loss.backward()
            # optimizer.step()
            # optimizer.zero_grad()
            
        print("Epoch: ", epoch, " Loss: ", history[-1])
        print("Validation RMSE: ", model.RMSE(validation=True))
        print("Training RMSE: ", model.RMSE(validation=False))
        # if history[-1]<0.09:
        #     break
    print('Final Loss ', history[-1])
    model.save(history)

    return history

In [233]:
X_valid = pd.concat(
    [test_df[dummies_names+['age_clipped']], X_test_scaled],
    axis=1,
    ignore_index=True,
)
y_valid = test_df["median_house_value"].reset_index(drop=True)

In [234]:
# data=CalHousingDataset(X_train_scaled, X_test_scaled)
model=CalHousing(6,50,X,y, X_valid,y_valid,13)
max_iter = 50000
num_epochs=500
lr=0.001
optimizer_ADAM = optim.Adam(model.model.parameters(),
                            lr=float(lr))

optimizer_LBFGS = optim.LBFGS(model.model.parameters(),
                              lr=float(0.5),
                              max_iter=max_iter,
                              max_eval=50000,
                              history_size=150,
                              line_search_fn="strong_wolfe",
                              tolerance_change=1.0 * np.finfo(float).eps)

fit(model, num_epochs, optimizer_ADAM)

################################  0  ##################################


Epoch:  0  Loss:  0.1963653266429901
Validation RMSE:  150131.431750867
Training RMSE:  151673.9960430337
################################  1  ##################################
Epoch:  1  Loss:  0.17288903892040253
Validation RMSE:  151520.91623238302
Training RMSE:  152863.1366218944
################################  2  ##################################
Epoch:  2  Loss:  0.15024423599243164
Validation RMSE:  153745.83275058033
Training RMSE:  155093.56597727272
################################  3  ##################################
Epoch:  3  Loss:  0.13761070370674133
Validation RMSE:  152620.78957258537
Training RMSE:  154147.3376537473
################################  4  ##################################
Epoch:  4  Loss:  0.14419853687286377
Validation RMSE:  152355.4460613585
Training RMSE:  153980.503952152
################################  5  ##################################
Epoch:  5  Loss:  0.142704576253891
Validation RMSE:  152119.96622653076
Training RMSE:  153711.797

KeyboardInterrupt: 

In [ ]:
X_valid = pd.concat(
    [test_df[dummies_names+['age_clipped']], X_test_scaled],
    axis=1,
    ignore_index=True,
)
y_valid = test_df["median_house_value"].reset_index(drop=True)
print("Training loss: ", model.validate_RMSE(X,y))
print("Validation loss: ",model.validate_RMSE(X_valid, y_valid))


Training loss:  184692.19701387145
Validation loss:  185961.63912275404


In [ ]:
model.model.eval()
with torch.no_grad():
    inputs=torch.tensor(X_valid.values.astype(np.float32),dtype=torch.float32).to(model.device)
    targets=torch.tensor(model.scale_targets(y_valid),dtype=torch.float32).to(model.device)
    preds=model.model(inputs)
    preds=model.scaler.inverse_transform(preds.cpu().numpy())
    loss=np.sqrt(np.mean((y_valid.values-preds)**2))
    print(loss)

155256.7839733259


In [ ]:
model.model.eval()
with torch.no_grad():
    inputs=torch.tensor(X.values.astype(np.float32),dtype=torch.float32).to(model.device)
    targets=torch.tensor(model.scale_targets(y),dtype=torch.float32).to(model.device)
    preds=model.model(inputs)
    preds=model.scaler.inverse_transform(preds.cpu().numpy())
    loss=np.sqrt(np.mean((y.values-preds)**2))
    print(loss)

156918.128669661
